In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Web scraping

## Stock names
Take all the name of stocks from siamchart

In [2]:
url = 'http://siamchart.com/stock-financial/'
html = requests.get(url).text
soup = BeautifulSoup(html)
tag_a= soup.find_all('a')
stocks_name = [tag_a[i].text.split('(')[0] for i in range(21,len(tag_a))]
print(stocks_name)

['24CS', '2S', '3K-BAT', '7UP', 'A', 'A5', 'AAI', 'AAV', 'ABICO', 'ABM', 'ACAP', 'ACC', 'ACE', 'ACG', 'ADB', 'ADD', 'ADVANC', 'AEONTS', 'AF', 'AFC', 'AGE', 'AH', 'AHC', 'AI', 'AIE', 'AIRA', 'AIT', 'AJ', 'AJA', 'AKP', 'AKR', 'ALL', 'ALLA', 'ALPHAX', 'ALT', 'ALUCON', 'AMA', 'AMANAH', 'AMARC', 'AMARIN', 'AMATA', 'AMATAV', 'AMC', 'AMR', 'ANAN', 'AOT', 'AP', 'APCO', 'APCS', 'APEX', 'APP', 'APURE', 'AQ', 'AQUA', 'ARIN', 'ARIP', 'ARROW', 'AS', 'ASAP', 'ASEFA', 'ASIA', 'ASIAN', 'ASIMAR', 'ASK', 'ASN', 'ASP', 'ASW', 'ATP30', 'AU', 'AUCT', 'AWC', 'AYUD', 'B', 'B52', 'BA', 'BAFS', 'BAM', 'BANPU', 'BAY', 'BBGI', 'BBIK', 'BBL', 'BC', 'BCH', 'BCP', 'BCPG', 'BCT', 'BDMS', 'BE8', 'BEAUTY', 'BEC', 'BEM', 'BEYOND', 'BFIT', 'BGC', 'BGRIM', 'BGT', 'BH', 'BIG', 'BIOTEC', 'BIS', 'BIZ', 'BJC', 'BJCHI', 'BKD', 'BKI', 'BLA', 'BLAND', 'BLESS', 'BLISS', 'BM', 'BOL', 'BPP', 'BR', 'BRI', 'BROCK', 'BROOK', 'BRR', 'BSBM', 'BSM', 'BTG', 'BTNC', 'BTS', 'BTW', 'BUI', 'BWG', 'BYD', 'CAZ', 'CBG', 'CCET', 'CCP', 'CEN', 'C

## Scraping Industry and Market Cap of each stock
Scrape these data from SET directly, I scraped them in Thai though

In [3]:
stocks_info = dict()
for stock in stocks_name:
    if '&' in stock :
        stock_name_temp = stock.replace('&','%26')
    else:
        stock_name_temp = stock

    url = 'https://classic.set.or.th/set/companyprofile.do?symbol='+stock_name_temp+'&ssoPageId=4&language=th&country=TH'
    html = requests.get(url).text
    soup = BeautifulSoup(html)
    inclass = soup.find_all(class_='col-xs-9 col-md-5')
    try:
        marketcap = float(inclass[3].text.replace(',','').strip())
        industry = inclass[5].text
    except:
        continue
    stocks_info[stock] = dict()
    stocks_info[stock]['industry'] = industry
    stocks_info[stock]['marketcap'] = marketcap
# Checking the data
pd.DataFrame(stocks_info).transpose()

,industry,marketcap
24CS,อสังหาริมทรัพย์และก่อสร้าง,1788.8
2S,สินค้าอุตสาหกรรม,1825.99
3K-BAT,สินค้าอุตสาหกรรม,5076.4
7UP,ทรัพยากร,4114.46
A,อสังหาริมทรัพย์และก่อสร้าง,4880.4
...,...,...
YGG,บริการ,4876.18
YONG,อสังหาริมทรัพย์และก่อสร้าง,1400.8
YUASA,สินค้าอุตสาหกรรม,1506.75
ZEN,เกษตรและอุตสาหกรรมอาหาร,5070.0


## Scraping Financial data
Only those that are needed to be used in calculating magic formula. From this step, we will need selenium library and your browser driver, I use microsoft edge, so I already have the driver in working directory. Since the website that contain this information is dynamic. So, this process will be around a hour. And each website of stocks have patterns which is good, otherwise, it would be impossible to scrape all the stock financial data.

In [5]:
driver = webdriver.Edge() #เปิด browser ขึ้นมา ใส่ตัวแปร driver เปิดครั้งเดียว
for stock in stocks_name:
    url = 'http://siamchart.com/stock-info/'+stock+'/' #ลิงค์ที่มีงบการเงินอยู่
    driver.get(url) #ให้ browser เปิดลิงค์ url
    html = driver.page_source #ดึง html จากเพจที่เปิดอยู่
    
    # ท่าเดิมครับ
    soup = BeautifulSoup(html)
    # งบไตรมาสล่าสุดอยู่ใน tag td @class = "remove_col"
    tdtag = soup.find_all('td',{'class':'remove_col'})
    
    #ดึงงบแต่ละตัวมาเลย
    #ถ้า error แปลว่า pattern เป็นแบบใน except
    try:
        liability_current = tdtag[85].text.split('(')[0]
        liability_total = tdtag[95].text.split('(')[0]
    except IndexError:
        
        #ถ้า error อีก แปลว่า ไม่มีงบในเว็บนี้
        try:
            ebit = tdtag[12].text.split('(')[0]
            cash = tdtag[18].text.split('(')[0]
            totalasset = tdtag[28].text.split('(')[0]
            liability_current = tdtag[36].text.split('(')[0]
            liability_total = tdtag[40].text.split('(')[0]
        except IndexError: 
            continue 
    ebit = tdtag[30].text.split('(')[0]
    cash = tdtag[45].text.split('(')[0]
    totalasset = tdtag[70].text.split('(')[0]
    
    #เซฟข้อมูลไว้ใน dictionary อันเดิมครับ
    try: #กรณีที่เป็นหุ้นที่ไม่มีในเว็บตลาดหลักทรัพย์ เราทิ้งตัวนั้นไปเลย
        stocks_info[stock]['ebit'] = ebit
        stocks_info[stock]['cash'] = cash
        stocks_info[stock]['totalasset'] = totalasset
        stocks_info[stock]['liability_current'] = liability_current
        stocks_info[stock]['liability_total'] = liability_total
    except KeyError:
        continue
print('Finished!')

Finished!


In [8]:
# Let's check the data
df = pd.DataFrame(stocks_info).transpose()
display(df.head())
print(f'There are total of stocks that missing data : {df.isna().any(axis=1).sum()}')
print(list(df[df.isna().any(axis=1)].index))# Let's check the data

,industry,marketcap,ebit,cash,totalasset,liability_current,liability_total
24CS,อสังหาริมทรัพย์และก่อสร้าง,1788.8,NaN,NaN,NaN,NaN,NaN
2S,สินค้าอุตสาหกรรม,1825.99,-2.34,104.66,"2,346.85",177.32,212.91
3K-BAT,สินค้าอุตสาหกรรม,5076.4,100.73,473.39,"3,958.31",104.88,648.69
7UP,ทรัพยากร,4114.46,-68.89,65.64,"4,559.75",141.73,640.67
A,อสังหาริมทรัพย์และก่อสร้าง,4880.4,300.14,165.19,"12,864.62","2,753.20","5,668.22"


There are total of stocks that missing data : 51
['24CS', 'AAI', 'AMARC', 'AMR', 'BBGI', 'BIS', 'BLESS', 'BTG', 'CEYE', 'CH', 'CHIC', 'CPANEL', 'DMT', 'F&D', 'FTI', 'GLORY', 'HENG', 'IND', 'ITNS', 'JDF', 'JR', 'JSP', 'KCC', 'KK', 'L&E', 'MENA', 'NSL', 'NV', 'PACO', 'PCC', 'PEACE', 'PIN', 'PLUS', 'PTC', 'SCAP', 'SECURE', 'SFT', 'SICT', 'SO', 'STECH', 'STP', 'SVT', 'TEGH', 'TEKA', 'TGE', 'TIDLOR', 'TIPH', 'TLI', 'TRV', 'WFX', 'YONG']


What happened after the code above run was there were some stocks have another different pattern which I was unable to identify. But after I look thorugh the website of these stocks, they're all stocks that don't have financial data in siamchart, so I will just drop them.

## Cleaning and preparing data
This process won't take us long, the only problem is that the financial data we collected are actually string not a numeric, so we won't be able to use this data to do calculation. Therefore, we are going to clean the data, only the number will be left, and change the data type to float.

In [12]:
df= pd.DataFrame(stocks_info).transpose()
# Change order of some columns
df = df.iloc[:, [1,0,2,3,4,5,6]]
# Drop missing data stocks
df = df.dropna()
columns_to_clean = ['ebit','cash','totalasset','liability_current','liability_total']
for column in columns_to_clean:
    df[column] = df[column].str.replace(',','').astype('float') 

# Applying magic formula
This is an another easy process, all we have to do just make some calculation finding ey and roc, but we have to filter stocks in financial sector out because magic formula won't invest on those.

In [13]:
# filter financial business out
cleaned_df = df[df['industry'] != 'ธุรกิจการเงิน']

cleaned_df['ey'] = cleaned_df['ebit'] / (cleaned_df['marketcap']+ cleaned_df['liability_total']- cleaned_df['cash'])
cleaned_df['roc'] = cleaned_df['ebit'] / (cleaned_df['totalasset'] - cleaned_df['liability_current'])
cleaned_df.head()

C:\Users\fnvin\AppData\Local\Temp\ipykernel_6324\3293095404.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['ey'] = cleaned_df['ebit'] / (cleaned_df['marketcap']+ cleaned_df['liability_total']- cleaned_df['cash'])
C:\Users\fnvin\AppData\Local\Temp\ipykernel_6324\3293095404.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['roc'] = cleaned_df['ebit'] / (cleaned_df['totalasset'] - cleaned_df['liability_current'])


,marketcap,industry,ebit,cash,totalasset,liability_current,liability_total,ey,roc
2S,1825.99,สินค้าอุตสาหกรรม,-2.34,104.66,2346.85,177.32,212.91,-0.00121,-0.001079
3K-BAT,5076.4,สินค้าอุตสาหกรรม,100.73,473.39,3958.31,104.88,648.69,0.01918,0.026140
7UP,4114.46,ทรัพยากร,-68.89,65.64,4559.75,141.73,640.67,-0.01469,-0.015593
A,4880.4,อสังหาริมทรัพย์และก่อสร้าง,300.14,165.19,12864.62,2753.20,5668.22,0.028906,0.029683
A5,3337.9,อสังหาริมทรัพย์และก่อสร้าง,112.04,174.25,1527.41,179.48,630.20,0.029532,0.083120


In [20]:
# Sort descdending by ey to do scoring
cleaned_df.sort_values('ey',ascending=False,inplace=True)
cleaned_df['ey_score'] = [x for x in range(1,len(cleaned_df)+1)]
# Sort descending by roc to do scoring
cleaned_df.sort_values('roc',ascending=False,inplace=True)
cleaned_df['roc_score'] = [x for x in range(1,len(cleaned_df)+1)]

# Combining ey_score and roc_score to receive a magic_score
cleaned_df['magic_score'] = cleaned_df[['ey_score','roc_score']].sum(axis=1)
cleaned_df.sort_values('magic_score',inplace = True)

# Top 30 
display(cleaned_df.head(30))
magic_formula = cleaned_df.head(30)
print('Top 30 stocks in magic formula are:')
print(list(magic_formula.index))

C:\Users\fnvin\AppData\Local\Temp\ipykernel_6324\183243455.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.sort_values('ey',ascending=False,inplace=True)
C:\Users\fnvin\AppData\Local\Temp\ipykernel_6324\183243455.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['ey_score'] = [x for x in range(1,len(cleaned_df)+1)]
C:\Users\fnvin\AppData\Local\Temp\ipykernel_6324\183243455.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pan

,marketcap,industry,ebit,cash,totalasset,liability_current,liability_total,ey,roc,ey_score,roc_score,magic_score
NATION,1057.67,บริการ,557.09,146.48,1304.01,237.50,141.69,0.529111,0.522349,5,6,11
RCL,23619.38,บริการ,15704.11,16625.75,54337.64,205.84,4309.17,1.3894,0.290109,1,14,15
LANNA,9922.49,ทรัพยากร,4638.13,3211.46,14790.25,294.38,1291.60,0.579576,0.319962,4,11,15
ECF,1707.89,สินค้าอุปโภคบริโภค,167.95,1314.47,656.66,239.87,26.48,0.399976,0.402961,7,8,15
CHOW,2656.0,สินค้าอุตสาหกรรม,1358.87,31.24,5023.12,432.51,233.26,0.475459,0.296011,6,13,19
BTNC,141.6,สินค้าอุปโภคบริโภค,-13.01,240.67,112.13,120.00,7.73,0.142435,1.653113,22,2,24
ESSO,50528.53,ทรัพยากร,17869.77,1305.04,93854.36,5333.73,6177.01,0.322556,0.201871,8,24,32
UVAN,6768.0,เกษตรและอุตสาหกรรมอาหาร,1066.79,1123.18,5107.65,552.52,849.18,0.164273,0.234195,16,19,35
ACC,1839.99,ทรัพยากร,203.64,507.66,832.25,44.37,71.12,0.1451,0.258466,20,17,37
VIH,5193.06,บริการ,686.75,543.52,3744.40,98.04,208.91,0.141352,0.188339,25,28,53


Top 30 stocks in magic formula are:
['NATION', 'RCL', 'LANNA', 'ECF', 'CHOW', 'BTNC', 'ESSO', 'UVAN', 'ACC', 'VIH', 'GPI', 'UMI', 'UPOIC', 'TAPAC', 'RJH', 'GYT', 'TOP', 'BANPU', 'IMH', 'KYE', 'PTTEP', 'BCP', 'SFP', 'AMATAV', 'BCH', 'BRI', 'NER', 'SKN', 'CHG', 'CPH']
